# <div align=center > Lab3 : Circuit Simulation Engine </div>

###  <div> Names and IDs:

<div align=center > Ahmed Hussien Abdelbary 1180482 </br> Sandra Hany Helmy 1180563 </div></div>

In [1]:
import numpy as np

In [2]:
class Component:
    def __init__(self,componentType,node1,node2,value,intialValue):
        self.componentType = componentType
        self.node1 = node1
        self.node2 = node2
        self.value = value
        self.intialValue = intialValue
         
    def __str__(self):
        return '''
        
        Component Type: {componentType}
        Node 1: {node1}
        Node 2: {node2}
        value: {value}
        Intial Value: {intialValue}
        
        '''.format(componentType = self.componentType,
                   node1 = self.node1 , 
                   node2=self.node2 , 
                   value = self.value , 
                   intialValue = self.intialValue)
    
    
    def __repr__(self):
        return self.__str__()

In [3]:
class Circuit:
    def __init__(self,simTime,numberOfIterations,components):
        self.simTime = simTime
        self.numberOfIterations = numberOfIterations
        self.components = components
    
    def __str__(self):
        return '''
        Simulation Time: {simTime} 
        Number of Iterations: {numberOfIterations}
        Components: {components}
        '''.format(simTime = self.simTime , 
                   numberOfIterations = self.numberOfIterations , 
                   components = self.components)
    
    def getNumberOfNodes(self):
        nodeSet = set()
        for component in self.components:
            nodeSet.add(component.node1)
            nodeSet.add(component.node2)
        return len(nodeSet)
    
    def getNumberofIndependetVoltageSources(self):
        voltageSourcesCount = 0
        for component in self.components:
            if component.componentType == "Vsrc" or component.componentType == "I":
                voltageSourcesCount+=1
        return voltageSourcesCount
    
    def getVoltageSourcesNodes(self):
        VSnodes = np.zeros(shape = (self.getNumberofIndependetVoltageSources(),2), dtype = int)
        idx = 0
        for component in self.components:
            if component.componentType == "Vsrc" or component.componentType == "I":
                VSnodes[idx,0] = component.node1
                VSnodes[idx,1] = component.node2
                idx+=1
        return VSnodes
    
    def getCurrentSourcesNodes(self):
        CSnodes = np.zeros(shape = (self.getNumberOfNodes(),2), dtype = int)
        CSValues = np.zeros(shape = (self.getNumberOfNodes(),1), dtype = float)
        idx = 0
        for component in self.components:
            if component.componentType == "Isrc":
                CSnodes[idx,0] = component.node1
                CSnodes[idx,1] = component.node2
                CSValues[idx,0] = component.value
                idx+=1
            elif component.componentType == "C":
                CSnodes[idx,0] = component.node1
                CSnodes[idx,1] = component.node2
                CSValues[idx,0] = (component.value/self.simTime) * component.intialValue
                idx+=1
        return CSnodes ,CSValues

In [4]:
def createComponent(data):
    data = data.split()
    if len(data) < 5:
        return
    else:
        return Component(componentType = data[0],
                         node1 = int(data[1][1]),
                         node2 = int(data[2][1]),
                         value = float(data[3]),
                         intialValue = float(data[4]))


def readConfigFile(path):
    circuit = Circuit(0,0,[])
    with open(path,'r') as file:
        for i,line in enumerate(file):
            if i== 0:
                circuit.simTime = float(line.strip())
            elif i == 1:
                circuit.numberOfIterations = int(line.strip())
            elif i > 1 :
                circuit.components.append(createComponent(line.strip()))
    circuit.components.pop(-1) #remove the last element flag
    return circuit

In [5]:
circuits = []
for i in range(1,9):
    circuits.append(readConfigFile(path = "testcases/"+str(i)+".txt"))

In [6]:
'''
    The Simulation Engine Solves the following Equation
                       Ax=z 
    where A is a matrix Composed of 4 sub-matrices G,B,C,D
    z is a matrix composed of 2 sub-matrices i,e
    x is a matrix composed of 2 sub-matrices v,j
'''   

class SimulationEngine:
    def __init__(self,circuit):
        self.circuit = circuit
        self.VSCount = circuit.getNumberofIndependetVoltageSources()
        self.nodeCount  = circuit.getNumberOfNodes()
        self.numberOfIterations = circuit.numberOfIterations
        self.simulationTime = circuit.simTime
        self.Z = None
        
        
#----------------------------------------------Construct A sub-matrices------------------------------------------------
    
    def __constructG(self):
        G = np.zeros(shape = (self.nodeCount,self.nodeCount) , dtype = float) 
        for component in self.circuit.components:
            if component.componentType == 'R':
                G[component.node1,component.node1] += 1/component.value
                G[component.node2,component.node2] += 1/component.value
                G[component.node1,component.node2] -= 1/component.value
                G[component.node2,component.node1] -= 1/component.value
            if component.componentType == 'C':
                G[component.node1,component.node1] += component.value/self.simulationTime
                G[component.node2,component.node2] += component.value/self.simulationTime
                G[component.node1,component.node2] -= component.value/self.simulationTime
                G[component.node2,component.node1] -= component.value/self.simulationTime
        return G[1:,1:]
            
    def __constructB(self):
        B = np.zeros(shape = (self.nodeCount,self.VSCount) , dtype = float)
        VSnodes = self.circuit.getVoltageSourcesNodes()
        for src_idx,source in enumerate(VSnodes):
            B[source[0],src_idx] = 1.0
            B[source[1],src_idx] = -1.0
        return B[1:]
    
    def __constructC(self):
        C = self.__constructB().T
        return C
    
    def __constructD(self):
        D = np.zeros(shape = (self.VSCount,self.VSCount) , dtype = float)
        
        return D 
    
#                  -------------------------------Concat Matrix A---------------------------------
    def __constructA(self):
        G = self.__constructG()
        B = self.__constructB()
        C = self.__constructC()
        D = self.__constructD()
        
        GB = np.concatenate((G,B),axis = 1)
        CD = np.concatenate((C,D),axis = 1)
        
        A = np.concatenate((GB,CD),axis = 0)
        return A
    
#-------------------------------------------------Construct Z sub-matrices---------------------------------------------

    def __constructI(self):
        I = np.zeros( shape = (self.nodeCount,1) , dtype = float)
        CSnodes,CSvalues = self.circuit.getCurrentSourcesNodes()
        for src_idx,node in enumerate(CSnodes):
            I[node[0],0] += CSvalues[src_idx,0] 
            I[node[1],0] -= CSvalues[src_idx,0]
        return I[1:]
    
        

    def __constructE(self):
        E = np.zeros( shape = (self.VSCount,1) , dtype = float)
        idx = 0
        for component in self.circuit.components:
            if(component.componentType == "Vsrc"):
                E[idx] = component.value
                idx+=1
            if(component.componentType == "I"):
                E[idx] = component.value / self.simulationTime
                idx+=1
        return E
    
    
#                   --------------------------------Concat Matrix z---------------------------------------
    def __constructZ(self):
        I = self.__constructI()
        E = self.__constructE()
        
        Z = np.concatenate((I,E), axis = 0)
        
        self.Z = Z
        
        return Z
    
    
        
#---------------------------------------Solve the linear System to find matrix X---------------------------------------
    def __calculateX(self):
        A = self.__constructA()
        Z = self.__constructZ()
        
        X = np.round(np.linalg.inv(A) @ Z , 13)
        return X
#-----------------------------------------------Update Component Values---------------------------------------------------------

    def __updateIntialValues(self,X):
        for component in self.circuit.components:
            if component.componentType == "C":
                if component.node1 != 0 and component.node2 != 0:
                    component.intialValue = X[component.node1 - 1] - X[component.node2 - 1]
                elif component.node1 != 0:
                    component.intialValue = X[component.node1 - 1]
                elif component.node2 != 0:
                    component.intialValue = X[component.node2 - 1]


#-----------------------------------------------print Solution---------------------------------------------------------
    
    def solve(self):
        for j in range(self.numberOfIterations):
            X = np.squeeze(self.__calculateX())
            self.__updateIntialValues(X)
            for i in range(1,self.nodeCount):
                print("V"+str(i)+" = "+str(X[i-1]))

            for i in range(self.VSCount):
                print("I_Vsrc"+str(i)+" = "+str(X[i+self.nodeCount-1]))
            print("___________End Iteration "+str(j+1)+" __________________")

        print("       ----------------------------------- End Simulation ---------------------------------------           \n")
        self.Z = None
        

In [8]:
# Part 1 Test Cases: First 4 Circuits e.g. 0:3

for circuit_idx,circuit in enumerate(circuits):
    if circuit_idx < 5 or circuit_idx == 6:
        simulationEngine = SimulationEngine(circuit= circuit)
        print("                                -------------------------Circuit: "+str(circuit_idx+1)+"------------------------       \n")
        simulationEngine.solve()

                                -------------------------Circuit: 1------------------------       

V1 = 10.0
V2 = 5.0
I_Vsrc0 = -1.6666666666667
___________End Iteration 1 __________________
       ----------------------------------- End Simulation ---------------------------------------           

                                -------------------------Circuit: 2------------------------       

V1 = 10.0
V2 = 1.2
I_Vsrc0 = -0.044
___________End Iteration 1 __________________
       ----------------------------------- End Simulation ---------------------------------------           

                                -------------------------Circuit: 3------------------------       

V1 = -8.0
V2 = 24.0
V3 = 20.0
I_Vsrc0 = -0.4
I_Vsrc1 = 0.1
___________End Iteration 1 __________________
       ----------------------------------- End Simulation ---------------------------------------           

                                -------------------------Circuit: 4------------------------